<a href="https://colab.research.google.com/github/calumrussell/fpl/blob/master/fplpoints_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
teams = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/teams.csv")
fixtures = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/fixtures.csv")
players_raw = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv")

In [3]:
gws = []
for i in range(1, 47):
    df = pd.read_csv(f'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw{i}.csv')
    gws.append(df)
players = pd.concat(gws)

In [4]:
##Removing all the data we don't need
players_filt = players[['name','value','fixture','opponent_team','total_points','minutes','transfers_balance','was_home']]
fixtures_filt = fixtures[['id', 'team_a','team_h']]
teams_filt = teams[['id', 'strength', 'short_name']]
players_raw_filt = players_raw[['id','element_type']].astype('int64')

In [5]:
players_filt['player_id'] = players_filt['name'].apply(lambda x: x.split("_")[2]).astype('int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
##Merging the fixtures onto players so we have information about the match and the player's team and opponent
merged = players_filt.merge(fixtures_filt, how='left', left_on='fixture', right_on='id')

In [7]:
##Merging position onto our players
merged = merged.merge(players_raw_filt, how='left', left_on='player_id', right_on='id' )

In [8]:
import numpy as np
##Create a row for the player's team
merged['team'] = np.where(merged['opponent_team'] == merged['team_a'], merged['team_h'], merged['team_a'])
##Create a row for player team difficulty
merged = merged.merge(teams_filt, how='left', left_on='team', right_on='id')
merged.rename(columns={'strength': 'team_diff', 'short_name': 'team_short'}, inplace=True)
##Create a row for opponent team difficulty
merged = merged.merge(teams_filt, how='left', left_on='opponent_team', right_on='id')
merged.rename(columns={'strength': 'opp_diff', 'short_name': 'opp_short'}, inplace=True)
##Create a row for difficulty difference
merged['diff_diff'] = merged['team_diff'] - merged['opp_diff']

In [9]:
merged.head()

,name,value,fixture,opponent_team,total_points,minutes,transfers_balance,was_home,player_id,id_x,team_a,team_h,id_y,element_type,team,id_x,team_diff,team_short,id_y,opp_diff,opp_short,diff_diff
0,Aaron_Cresswell_376,50,8,11,0,90,0,True,376,8,11,19,376,2,19,19,2,WHU,11,5,MCI,-3
1,Aaron_Lennon_430,50,3,16,1,6,0,True,430,3,16,5,430,3,5,5,3,BUR,16,3,SOU,0
2,Aaron_Mooy_516,50,7,18,0,0,0,False,516,7,4,18,516,3,4,4,2,BHA,18,3,WAT,-1
3,Aaron_Ramsdale_494,45,2,15,2,90,0,True,494,2,15,3,494,1,3,3,2,BOU,15,3,SHU,-1
4,Aaron_Wan-Bissaka_122,55,9,6,8,90,0,True,122,9,6,12,122,2,12,12,4,MUN,6,4,CHE,0


In [10]:
##Was_home to int value
merged['is_home'] = merged['was_home'].apply(lambda x: 1 if x else 0)

Here, we perform cleaning to help improve the accuracy of our results.

First, we cut all entries off below zero, and add one to the points in every row. We do this because the data is heavily right skewed, and we need to take the log of points (and we can't have log(0)).

We will be modelling results with simple regression, and if we take the histogram of total points (not included here to make this post shorter), we see that we have lots of values around zero, and a few very large values. Most regression models tend to struggle with these skewed distributions, an assumption of OLS is normally distributed errors/linear parameters, so we need to unskew our data. In simple numeric terms: if we have a points total of 5 and a points total of 25, the log difference between these two scores is ~1x as opposed to 5x on the normal scale. We will explain more about the model choice later.

Second, we remove matches where players didn't play. If we included these results, we would get a misleading picture of model accuracy, and would be predicting something we with certainty: if a player doesn't play, they score zero points. An interesting alternative choice here is to remove matches where players played less than 90 minutes (i.e. didn't start). We explore this later.

In [11]:
import numpy as np
cleaned = merged.copy(deep=True)

cleaned.loc[cleaned['total_points'] < 0, 'total_points'] = 0
cleaned['total_points'] = (cleaned['total_points'] + 1).astype('float32')
cleaned['log_points'] = np.log(cleaned['total_points'])

cleaned = cleaned.drop(cleaned[cleaned.minutes == 0].index)

In [12]:
X = cleaned[['log_points']].astype('float32')
y = pd.DataFrame()
y['team_short'] = cleaned['team_short']
y['opp_short'] = cleaned['opp_short']
y = pd.get_dummies(y, columns=['team_short'])
y = pd.get_dummies(y, columns=['opp_short'])

Our first model looks at points scored as a function of team and opponent strength. The purpose of this model is to familiarize you with: how to build a model, and the output.

The only input here is the identity of the teams. This may seem a bit odd but what we are actually modelling here are the strength of teams, in terms of log points, offensively and defensively.

We include an intercept for this model but we do not do this for all models. In this case, an intercept makes sense: it represents the average points scored exclusive of team/opponent strengths. But, in other cases, including an intercept made it difficult to interpret the parameters, so we do not include it every time (this is likely due to multicollinearity amongst independent variables, I am sure that a skilled practitioner could tease these differences out but it was beyond me).

In [13]:
import statsmodels.api as sm
y = sm.add_constant(y)
model = sm.OLS(X, y).fit()
print(model.summary())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:             log_points   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     13.58
Date:                Thu, 01 Oct 2020   Prob (F-statistic):           8.33e-83
Time:                        00:35:55   Log-Likelihood:                -9396.5
No. Observations:               10313   AIC:                         1.887e+04
Df Residuals:                   10274   BIC:                         1.915e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.0577      0.005    195.

We won't look at model performance here. Again, this is a simple (but useful) starter but we will examine the meaning of our model summary.

Players that were part of a strong side - such as Liverpool - scored more points, and players that faced a weak opponent - such as Norwich - also scored more points. Not surprising but there is a lot of information outside the extreme cases. One example that I noticed is that Arsenal has a surprisingly weak offence and a surprisingly strong defence.

Also, this data is ex-post: we are using the outcomes of matches to tell us which teams are strongest. The model is informative, not predictive. A model could be built using trailing strength parameters but these models are often quite tricky to worth with: we would need to control for fixture difficult, and to lots of validation to work out how many matches we need to gain information. In this case though, this model is just informative.

The next model is our benchmark model. All factors that show predictive ability are included: assume that we can have accurate team strength parameters and lineups, what kind of accuracy could we produce?

A key decision was model choice: OLS with log points, GLM with log points, or some other GLM with raw points that somehow worked.

In practice, there was little difference between OLS and GLM with log points. A Poisson GLM did outperform in some cases but this difference seemed marginal. The last option didn't work.

In [14]:
X = cleaned[['log_points']].astype('float64')
y = cleaned[['value', 'is_home', 'diff_diff', 'element_type', 'minutes']].astype('float64')
y['team_short'] = cleaned['team_short']
y['opp_short'] = cleaned['opp_short']

y = pd.get_dummies(y, columns=['is_home'])
y = pd.get_dummies(y, columns=['team_short'])
y = pd.get_dummies(y, columns=['opp_short'])
y = pd.get_dummies(y, columns=['element_type'])

In [15]:
import statsmodels.api as sm
model = sm.OLS(X, y).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_points   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     72.11
Date:                Thu, 01 Oct 2020   Prob (F-statistic):               0.00
Time:                        00:35:55   Log-Likelihood:                -8260.8
No. Observations:               10313   AIC:                         1.661e+04
Df Residuals:                   10268   BIC:                         1.694e+04
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
value                0.0073      0.001  

This model includes:
* Player value before the match
* The difference in the team difficulty i.e. the difficulty advantage over the opponent
* Minutes played, this is kind of data leakage but assumes we can predict who will play, which isn't totally unreasonable
* Whether the player's team is home or away (we include dummies so we can see the actual parameter value for both).
* Dummies for the player's team and opponent's team
* The player's position

We are clearly having problems with multicollinearity here but this model is going purely for: let's just see what the limit on prediction is. 

In particular, the values for the home/away dummy, team/opponent dummies, and positions have lost all meaning.

For example, the parameter value for Liverpool's team dummy is negative. Logically, being on Liverpool helps a player score more points...so what is happening?

The likely source of the issue is correlation with one or more other factors in the model: team strength is correlated with another independent variable (or more than one).

In this case, the culprit is likely to be player value (which is a very strong parameter): playing for Liverpool did help score more points but most of Liverpool's players had high prices, so our parameter estimate for team strength is reduced.

To see this more clearly, look at the value of the dummy for Manchester City: even more negative, probably due to their players having high valuations with fairly poor results (note: we don't care about this, the aim of FPL is to maximise total points, not points per value).

I did test this theory, with a model including just team/opponent dummies and value, and this relationship held...but this isn't conclusive. Hopefully, this section highlights the difficulty in interpreting model parameters when there is correlation between independent variables/multicollinearity (we don't do this here but it is possible although often difficult to remove these effects).

In [16]:
from sklearn.metrics import mean_squared_error
print(model.aic, mean_squared_error(X, model.predict()))

16611.581448981335 0.2905841727702081


Multicollinearity doesn't change our predictions and this benchmark model performed okay: MSE of 0.29, this is log points though so this is 1.32 of "real" points.

In [17]:
only_90 = cleaned.drop(cleaned[cleaned.minutes != 90].index)

In [18]:
X = only_90[['log_points']].astype('float64')
y = only_90[['value', 'is_home', 'diff_diff', 'element_type']].astype('float64')
y['team_short'] = only_90['team_short']
y['opp_short'] = only_90['opp_short']

y = pd.get_dummies(y, columns=['is_home'])
y = pd.get_dummies(y, columns=['team_short'])
y = pd.get_dummies(y, columns=['opp_short'])
y = pd.get_dummies(y, columns=['element_type'])

In [19]:
import statsmodels.api as sm
model = sm.OLS(X, y).fit()

In [20]:
print(model.aic, mean_squared_error(X, model.predict()))

10779.017978437227 0.34405237393661975


Removing those players who didn't start, we actually get a worse result in terms of mean squared error. We might expect the opposite here: surely, it would be easier to predict outcomes when all players were on the pitch for the same length of time. But, given that it is harder, it seems likely that the previous results in our benchmark model were due to the fact that most players in our sample didn't score many points: most starters in a match don't score many points, and most subs don't score many points either.

We also got a lower AIC above, this is likely to due to features of the minutes variable. I didn't examine this but it is possibly correlated to other factors, again most likely value, in such a way that removing the minutes variable altogether improves the model quality (although not predictions).

In [21]:
df = pd.DataFrame({"actual":X['log_points'].values, "predicted": model.predict()})
bins = np.linspace(df.actual.min(), df.actual.max(), 7)
groups = df.groupby(np.digitize(df.actual, bins))
groups.mean()

,actual,predicted
1,0.000000,1.070184
2,0.693147,1.177833
3,1.149512,1.317539
4,1.918399,1.346409
5,2.346137,1.420521
6,2.814387,1.557345
7,3.218876,1.345473


We can represent this failure to predict higher values in the above table. We have binned our predictions, and taken the mean of each bin over the range of actual values.

Our model is predicting values that do increase but which appear to be centered around 1.3-1.5. Nowhere near what we see in reality.

To say this simply: it is easy to get a small-sounding error when the majority of your sample is below 5 points. You just always say: 1.3 points, and you will be close most of the time. Our model isn't doing this, but the error we have sounds more impressive than it is.

Additionally, our sample still has a chunky mode around these low values. Our model performs well here but terribly everywhere else. Our attempts to unskew this data were not wholly successful.

Statistical theory suggests that a general linear model using alternate parametric assumptions - for example, Poisson regression - will outperform OLS when we have errors that increase. But no model, that I tried, did actually outperform OLS. However, there other options.

The most likely option would be to recognise that we are modelling a mixture of distributions. We could model this non-parameterically. Or we could split our model by position: either constructing a seperate model per position (if we had more detailed player-level stats) or a mixed effects model with position as fixed effects. 

An interesting alternative, in theory although it possibly wouldn't work in practice, would be a state space model. We model a normal match, we model a "points haul" match where a player scores lots of points, and then make our total points prediction on points in each prediction weighted by the probability of being in each state. In reality, the probability of a points haul isn't identically distributed between players so a position-based model would likely be a more worthwhile first iteration.

In [22]:
X = cleaned[['log_points']].astype('float64')
y = cleaned[['value', 'is_home']].astype('float64')

y = pd.get_dummies(y, columns=['is_home'])

In [31]:
import statsmodels.api as sm
model = sm.OLS(X, y).fit()

In [24]:
from sklearn.metrics import mean_squared_error
print(model.aic, mean_squared_error(X, model.predict()))

18717.08990510335 0.3593148284219615


Reusing our dataset with substitute players and creating a very simple model: player value and whether the player is at home. MSE was 0.35 against 0.29 versus our benchmark model. In other words, the value of adding lots of information above just player value and home/away is 0.06 log points. Our AIC is higher, which also suggests that adding more parameters did improve accuracy.

In [25]:
X = cleaned[['log_points']].astype('float64')
y = cleaned[['value', 'is_home', 'diff_diff', 'element_type', 'team_diff', 'opp_diff', 'minutes']].astype('float64')

##Team and opponent difficulty are actual ordinal values, and the scale itself doesn't have meaning
y = pd.get_dummies(y, columns=['is_home'])
y = pd.get_dummies(y, columns=['team_diff'])
y = pd.get_dummies(y, columns=['opp_diff'])
y = pd.get_dummies(y, columns=['element_type'])

In [26]:
import statsmodels.api as sm
model = sm.OLS(X, y).fit()

In [27]:
from sklearn.metrics import mean_squared_error
print(model.aic, mean_squared_error(X, model.predict()))

16710.25186098493 0.295203993712952


We now propose a more realistic ex-ante model with less precise measures of team and opponent strength, and assuming some ability to predict who will play (proxied by the minutes variable).

Our results are, surprisingly, most of the way to our benchmark model (which had an MSE of 0.29). But, somewhat interestingly given what we found with AIC dropping with minutes being excluded, this appears to be due largely to our minutes variable.

In [28]:
X = only_90[['log_points']].astype('float64')
y = only_90[['value', 'is_home', 'diff_diff', 'element_type', 'team_diff', 'opp_diff']].astype('float64')

##Team and opponent difficulty are actual ordinal values, and the scale itself doesn't have meaning
y = pd.get_dummies(y, columns=['is_home'])
y = pd.get_dummies(y, columns=['team_diff'])
y = pd.get_dummies(y, columns=['opp_diff'])
y = pd.get_dummies(y, columns=['element_type'])

In [29]:
import statsmodels.api as sm
model = sm.OLS(X, y).fit()

In [30]:
from sklearn.metrics import mean_squared_error
print(model.aic, mean_squared_error(X, model.predict()))

10852.34024666836 0.3519681429568843


If we use our dataset that only has players who started and drop the minutes variable, we see a very marginal increase in MSE relative to our model that only had value and home/away: MSE of 0.351 vs 0.359.

**What conclusions can be draw?**

1. Points scored is heavily skewed. Log transform improves model accuracy but there is still more improvement out there.
2. Points scored is multi-modal. Other analysis I have performed suggest that points distribution varies by player position, a mixed effects model could be more appropriate or seperate models for each position.
3. We have not tested the construction of a proper ex-ante team strength model but the very basic team strength model offered by FPL suggests that team strength does improve model accuracy but in a clearly bounded fashion. Other analysis I have performed suggests there are interesting correlations between offensive/defensive strength and position: in other words, some teams score points through defending/clean-sheets.
4. Player value/home advantage clearly have predictive value.
5. Player-level stats are likely crucial to model accuracy. Even our ex-post model with perfect estimates of team strength suggests that there is still significant amounts of variation that is distinct from team strength. This might be surprising: don't players at the top clubs score all the points? In the top 3, maybe but there is clearly stil more to this variable. Our last model has MSE of 1.4 points (non-log) but there is still a lot left to explain outside of low points outcomes.